# Research Uniswap market data

In [221]:
import requests
import pandas as pd
from datetime import datetime
import plotly.express as px

## UNI cex Trading volumes

In [220]:
categ = requests.get('https://api.coingecko.com/api/v3/coins/categories/list')
categ.json()
#'category_id': 'centralized-exchange-token-cex',
  #'name': 'Centralized Exchange (CEX)'},
#'category_id': 'decentralized-exchange',
  #'name': 'Decentralized Exchange (DEX)'

[{'category_id': 'aave-tokens', 'name': 'Aave Tokens'},
 {'category_id': 'algorand-ecosystem', 'name': 'Algorand Ecosystem'},
 {'category_id': 'analytics', 'name': 'Analytics'},
 {'category_id': 'arbitrum-ecosystem', 'name': 'Arbitrum Ecosystem'},
 {'category_id': 'arbitrum-nova-ecosystem', 'name': 'Arbitrum Nova Ecosystem'},
 {'category_id': 'artificial-intelligence', 'name': 'Artificial Intelligence'},
 {'category_id': 'asset-backed-tokens', 'name': 'Asset-backed Tokens'},
 {'category_id': 'asset-manager', 'name': 'Asset Manager'},
 {'category_id': 'augmented-reality', 'name': 'Augmented Reality'},
 {'category_id': 'automated-market-maker-amm',
  'name': 'Automated Market Maker (AMM)'},
 {'category_id': 'avalanche-ecosystem', 'name': 'Avalanche Ecosystem'},
 {'category_id': 'axie-infinity', 'name': 'Axie Infinity'},
 {'category_id': 'big-data', 'name': 'Big Data'},
 {'category_id': 'binance-launchpool', 'name': 'Binance Launchpool'},
 {'category_id': 'binance-smart-chain', 'name': 'B

## UNI dex Trading volume 

## UNI Prices

In [222]:
uni_chart = requests.get(
    "https://api.coingecko.com/api/v3/coins/uniswap/market_chart?vs_currency=usd&days=1000").json()

In [223]:
uni_prices = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_chart['prices']], 
    'prices': [value[1] for value in uni_chart['prices']]
})

In [225]:
px.line(uni_prices, x='date', y='prices')

In [247]:
#На первый взгляд ходит вместе с основными валюлтами

## UNI total supply

In [226]:
uni_total_supply = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_chart['total_volumes']], 
    'total_supply': [market_cap[1] / price[1] for price, market_cap in zip(uni_chart['prices'], 
                                                                    uni_chart['total_volumes'])]
})

In [227]:
px.line(uni_total_supply, x='date', y='total_supply')

In [246]:
# В основном имеет относительно постоянный total_supply, что логично.

## Uniswap TVL 

In [228]:
uniswap= requests.get('https://api.llama.fi/protocol/uniswap').json()

In [234]:
tvls = uniswap['chainTvls']

In [235]:
tvl = None
for i, key in enumerate(tvls.keys()):
    tmp = pd.DataFrame(tvls[key]['tvl']).rename(columns={'totalLiquidityUSD': key})
    if i == 0:
        tvl = tmp
    else: 
        tvl = tvl.merge(tmp, on='date', suffixes=('', key), how='outer')
    
tvl['date'] = tvl.date.apply(lambda x: datetime.utcfromtimestamp(x))

### Ethereum

In [236]:
px.line(tvl[tvl.date > '2022-01-01'], x='date', y=[x for x in tvl.columns if x != 'date'])

### Optimism, Polygon, Arbirtum

In [237]:
tvl_No_ETH = tvl.drop('Ethereum', axis=1)

In [238]:
px.line(tvl_No_ETH[tvls_df_No_ETH.date > '2021-09-01'], x='date', 
        y=[x for x in tvl_No_ETH.columns if x != 'date'])

### Celo 

In [245]:
tvl_just_Celo=tvls_df_No_ETH.drop(['Optimism','Polygon','Arbitrum'],axis=1)
px.line(tvl_just_Celo[tvl_just_Celo.date > '2022-07-09'], x='date', y=[x for x in tvl_just_Celo.columns if x != 'date'])

### Uniswap Trading Volumes 

In [244]:
uniswap_volume_json = requests.get('https://api.coingecko.com/api/v3/exchanges/uniswap/volume_chart?days=500').json()
uniswap_volume_df = pd.DataFrame({
        'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uniswap_volume_json], 
        'value': [float(value[1]) for value in uniswap_volume_json],
    })
px.line(uniswap_volume_df, x='date', y='value')